로지스틱 회귀 모델 구현

In [573]:
# from sklearn import svm,metrics
# from sklearn.neighbors import KNeighborsClassifier # 븐류
# from sklearn.neighbors import KNeighborsRegressor # 회귀- 예측
from sklearn.linear_model import LinearRegression # 선형회귀
from sklearn.preprocessing import PolynomialFeatures # 다항특성을 만들어주는 라이브러리
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib
import glob, os.path, re

matplotlib.rcParams['axes.unicode_minus'] = False  # 마이너스 표시 해결
# 한글설정
matplotlib.rcParams['font.family'] = 'Malgun Gothic' # windows 사용자
# matplotlib.rcParams['font.family'] = 'AppleGothic Gothic' # Mac사용자
matplotlib.rcParams['font.size'] = '10' # 글자크기

In [574]:
import pandas as pd
df = pd.read_csv('https://bit.ly/fish_csv')
df

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340
...,...,...,...,...,...,...
154,Smelt,12.2,12.2,13.4,2.0904,1.3936
155,Smelt,13.4,12.4,13.5,2.4300,1.2690
156,Smelt,12.2,13.0,13.8,2.2770,1.2558
157,Smelt,19.7,14.3,15.2,2.8728,2.0672


In [575]:
df.describe()

,Weight,Length,Diagonal,Height,Width
count,159.000000,159.000000,159.000000,159.000000,159.000000
mean,398.326415,28.415723,31.227044,8.970994,4.417486
std,357.978317,10.716328,11.610246,4.286208,1.685804
min,0.000000,8.400000,8.800000,1.728400,1.047600
25%,120.000000,21.000000,23.150000,5.944800,3.385650
50%,273.000000,27.300000,29.400000,7.786000,4.248500
75%,650.000000,35.500000,39.650000,12.365900,5.584500
max,1650.000000,63.400000,68.000000,18.957000,8.142000


In [576]:
# 분류하는 형태 7가지
df['Species'].unique()

array(['Bream', 'Roach', 'Whitefish', 'Parkki', 'Perch', 'Pike', 'Smelt'],
      dtype=object)

In [577]:
# str 문자 타입이 존재 한다면
# - 제거를 나중에 타입을 df 로 합칠지
# - 원핫인코딩을 하든지

In [578]:
# pandas 원핫인코딩 함수
df_dumm =pd.get_dummies(df['Species'])
df_dumm.head(36)

,Bream,Parkki,Perch,Pike,Roach,Smelt,Whitefish
0,True,False,False,False,False,False,False
1,True,False,False,False,False,False,False
2,True,False,False,False,False,False,False
3,True,False,False,False,False,False,False
4,True,False,False,False,False,False,False
5,True,False,False,False,False,False,False
6,True,False,False,False,False,False,False
7,True,False,False,False,False,False,False
8,True,False,False,False,False,False,False
9,True,False,False,False,False,False,False


In [579]:
df['Species'].head(2)

0    Bream
1    Bream
Name: Species, dtype: object

In [580]:
df['Species'].value_counts()

Species
Perch        56
Bream        35
Roach        20
Pike         17
Smelt        14
Parkki       11
Whitefish     6
Name: count, dtype: int64

In [581]:
# str타입이 컬럼에 있는지 확인
# 머신러닝을 돌릴 수가 없음
# 문자 - 삭제 or 코드로 변경 하는 지에 대해서 결정
# - 1. 문자 아스키 코드 변경 - 표준점수
# - 2. 원핫인코딩 방식

# object(str)  - result, target 은 문자가 올 수가 있음 

In [582]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Species   159 non-null    object 
 1   Weight    159 non-null    float64
 2   Length    159 non-null    float64
 3   Diagonal  159 non-null    float64
 4   Height    159 non-null    float64
 5   Width     159 non-null    float64
dtypes: float64(5), object(1)
memory usage: 7.6+ KB


In [583]:
# 데이터 전처리 과정 - data, result 분리
# 데이터 전처리 과정 - data, result 분리
# - 데이터 전처리 - 표준화 작업
# 데이터 학습시키기
# 데이터 정답률

# 정답률이 낮다면
# - 데이터 전처리 후 다시 학습시키기

판다스 형태 전처리 - data, result 표준화 작업

In [584]:
# 데이터 전처리 과정 - data, result 분리
d_input = df.iloc[:,1:].to_numpy()
d_target = df['Species'].to_numpy()

In [585]:
# d_target = df['Species']
# d_input = df.iloc[:,1:]
type(d_target)

type(d_input)

numpy.ndarray

In [586]:
# 데이터 전처리 과정 - data, result 분리
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
     d_target, d_input
)

In [587]:
# - 데이터 전처리 - 표준화 작업
# 타입이 numpy 어야함
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_input)


ValueError: could not convert string to float: 'Roach'

In [ ]:
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

ValueError: could not convert string to float: 'Bream'

전처리 완성

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_scaled,train_target)


NameError: name 'train_scaled' is not defined

In [ ]:
# 정답률 확인
print(lr.score(train_scaled,train_target))
print(lr.score(test_scaled,test_target))

NameError: name 'train_scaled' is not defined

In [ ]:
### 반복 횟수를 늘려서 정답률 높이는 작업
lr = LogisticRegression(C=20,max_iter=1000)
lr.fit(train_scaled,train_target)
print(lr.score(train_scaled,train_target))
print(lr.score(test_scaled,test_target))

NameError: name 'train_scaled' is not defined

In [ ]:
# 로지스틱 회귀 분류 모델을 완성

In [ ]:
# 경사하강법 - 머신러닝 튜닝 : 기존에 훈련된 모델에서 다시 훈련
from sklearn.linear_model import SGDClassifier
sc = SGDClassifier(loss='log_loss', max_iter=10,random_state=42)

sc.fit(train_scaled,train_target)

print(sc.score(train_scaled,train_target))
print(sc.score(test_scaled,test_target))

In [ ]:
classes = np.unique(train_input)
classes

array(['Bream', 'Parkki', 'Perch', 'Pike', 'Roach', 'Smelt', 'Whitefish'],
      dtype=object)

In [ ]:
# 기존에 있는 거에 추가

# 데이터 일부를 가져와서 훈련을 했기에, target을 맞추는데, 전체적인 target을 알지 못함
# 전체 target이 무엇인지 알려줘ㅇ함
# classes 추가
sc.partial_fit(train_scaled,train_target,classes=classes)

print(sc.score(train_scaled,train_target))
print(sc.score(test_scaled,test_target))

In [ ]:
# partial_fit 반복을 시키면 효율이 좋아짐
train_score = []
test_score = []
for _ in range(0,300):
     sc.partial_fit(train_scaled,train_target,classes=classes)
     train_score.append(sc.score(train_scaled,train_target))
     test_score.append(sc.score(test_scaled,test_target))

In [ ]:
plt.plot(train_score)
plt.plot(test_score)

In [ ]:
sc = SGDClassifier(loss='log_loss',max_iter=100,tol=None,random_state=42) # 반복횟수 100번에 멈춤
sc.fit(train_scaled,test_scaled)
print(sc.score(train_scaled,train_target))
print(sc.score(test_scaled,test_target))
